In [30]:
#!pip install seleinum
#!pip install BeautifulSoup
#!pip install openpyxl
from bs4 import BeautifulSoup
from selenium import webdriver
from openpyxl import Workbook

In [31]:
def makeURL(_url, _stn, _yy, _obs, _x='14', _y='2'):
    URL = _url + '?' + 'stn=' + _stn +'&yy=' + _yy + '&obs=' + _obs + '&x=' + _x + '&y=' + _y
    return URL

In [32]:
def openWebsite(_url):
    driver = webdriver.Chrome('./chromedriver')
    driver.get(_url)
    return driver

In [33]:
def getHtml(driver):
    soup = BeautifulSoup(driver.page_source,'html.parser')
    return soup

In [34]:
# def setArea(driver, area):
#     driver.find_element_by_xpath("//select[@id='observation_select1']/option[@value='" + area + "']").click()
#     print(driver)
#     return driver

# def setYear(driver, year):
#     driver.find_element_by_xpath("//select[@id='observation_select2']/option[@value='" + year + "']").click()
#     print(driver)
#     return driver

# def setObs(driver, obs):
#     driver.find_element_by_xpath("//select[@id='observation_select3']/option[@value='" + obs + "']").click()
#     print(driver)
#     return driver

In [35]:
def scrappingData(url, area, year, obs, dict_weather):
    if type(dict_weather) != type({}):
        print("put dict_weather argument as a dictionary type")
        return
    if dict_weather.get(year) == None:
        dict_weather[year] = {}
    if dict_weather[year].get(area) == None:
        dict_weather[year][area] = {}
    
    if dict_weather[year][area].get('1월') != None:
        days_by_month = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
        if int(year) % 4 == 0:
            days_by_month[1] += 1

        for m in range(12):
            month = str(m+1)+'월'
            dict_weather[year][area][month] = {}
            for d in range(days_by_month[m]):
                day = str(d+1)+'일'
                dict_weather[year][area][month][day] = []
    
    URL = makeURL(url, area, year, obs)
    driver = openWebsite(URL)
    soup = getHtml(driver)
    
    weathers = soup.select('#content_weather > table > tbody > tr > td')
    cnt = 1
    num = 12*31
    for weather in weathers:
        if num == 0:
            break
        if weather.text.find('일') != -1:
            day = weather.text
            continue
        month = str(cnt) + '월'

        cnt += 1
        num -= 1
        if (cnt == 13):
            cnt = 1
        
        if dict_weather[year][area][month].get(day) == None:
            continue
        dict_weather[year][area][month][day].append(weather.text)
    driver.quit()
        

In [49]:
def scrappingAllData(url, areas, years, obss, obs_str):
    for year in years:
        #해당 연도에 대한 날씨 정보 딕셔너리에 저장
        dict_weather = {}
        for area in areas:
            for obs in obss:
                scrappingData(url, area, year, obs, dict_weather)
        
        days_by_month = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
        if int(year) % 4 == 0:
            days_by_month[1] += 1
        
        print(dict_weather)
        
        #해당 연도에 대한 날씨 정보 파일에 저장
        wb = Workbook()
        ws = wb.active
        ws.title = "weather"+str(year)
        column_list = []
        column_list.append('날짜')
        column_list.append('지역')
        for s in obs_str:
            column_list.append(s)
        ws.append(column_list)
        
        for month in range(12):
            for day in range(days_by_month[month]):
                for area in areas:
                    row_list = dict_weather[year][area][str(month+1)+'월'][str(day+1)+'일']
                    if day+1 < 10:
                        daystring = '0'+str(day+1)
                    else:
                        daystring = str(day+1)
                    if month+1 < 10:
                        monthstring = '0'+str(month+1)
                    else:
                        monthstring = str(month+1)
                    row_list.insert(area)
                    row_list.insert(yeasr+'-'+monthstring+'-'+daystring)
                    ws.append(row_list)
        
        wb.save(filename=ws.title)

In [50]:
cities = ["seoul", "incheon", "suwon", "cheongju", "daejeon", "gwangju", "daegu"\
          ,"pohang", "ulsan", "busan", "changwon"]
cities_num = ["108", "112", "119", "131", "133", "156", "143", "138", "152", "159", "155"]
url = 'https://www.weather.go.kr/weather/climate/past_table.jsp'
yy = ["2020", "2019", "2018"]
obs = ["07", "10", "08", "21", "06", "12", "35", "59"]
obs_str = ["평균기온", "최저기온", "최고기온", "강수량", "평균풍속", "상대습도", "일조시간", "운량"]

In [51]:
scrappingAllData(url, cities_num, yy, obs, obs_str)

{'2020': {'108': {'1월': {'1일': ['8.9'], '2일': ['7.9'], '3일': ['0.0'], '4일': ['2.1'], '5일': ['3.9'], '6일': ['9.8'], '7일': ['10.0'], '8일': ['9.6'], '9일': ['1.6'], '10일': ['3.1'], '11일': ['3.6'], '12일': ['3.6'], '13일': ['1.4'], '14일': ['1.0'], '15일': ['0.1'], '16일': ['5.0'], '17일': ['4.8'], '18일': ['2.1'], '19일': ['4.8'], '20일': ['2.3'], '21일': ['2.9'], '22일': ['7.8'], '23일': ['3.4'], '24일': ['3.0'], '25일': ['7.4'], '26일': ['5.8'], '27일': ['9.5'], '28일': ['7.3'], '29일': ['6.6'], '30일': ['4.0'], '31일': ['4.8']}, '2월': {'1일': ['3.4'], '2일': ['6.1'], '3일': ['1.5'], '4일': ['3.9'], '5일': ['0.0'], '6일': ['0.9'], '7일': ['8.9'], '8일': ['2.5'], '9일': ['2.8'], '10일': ['1.1'], '11일': ['2.3'], '12일': ['9.9'], '13일': ['9.1'], '14일': ['2.1'], '15일': ['5.8'], '16일': ['8.9'], '17일': ['6.5'], '18일': ['0.0'], '19일': ['6.0'], '20일': ['2.5'], '21일': ['9.3'], '22일': ['5.9'], '23일': ['0.9'], '24일': ['7.0'], '25일': ['7.5'], '26일': ['5.3'], '27일': ['2.6'], '28일': ['9.8'], '29일': ['3.0']}, '3월': {'1일': ['8.1'], '

TypeError: insert expected 2 arguments, got 1

{2012: {108: {1: 1}, 109: {0: 0}}}
